In [1]:
import json
import nltk

### 将文本加载为json格式

In [2]:
input_file = 'tweets.txt'
with open(input_file, 'r') as f:
    lines = f.readlines()
items = [json.loads(x) for x in lines]

### 定义预处理类
* 大写转小写，去除尾部网址
* 分词
* 去除标点符号和停用词

In [3]:
class Preprocess:
    def __init__(self):
        self.punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', '...']
        self.stop_words = set(nltk.corpus.stopwords.words('english'))
    def __call__(self,query):
        query = query.lower().split('http')[0]
        query = nltk.word_tokenize(query)
        query = [x for x in query if x not in self.punctuations and x not in self.stop_words]
        return query

In [4]:
preprocess = Preprocess()
tweets = [preprocess(x['text']) for x in items]

### 统计词频

In [5]:
dictionary = {}
for i, tweet in enumerate(tweets):
    freq_dist = nltk.FreqDist(tweet)
    for word, freq in freq_dist.items():
        if word in dictionary:
            dictionary[word].add(i)
        else:
            dictionary[word] = set([i])

In [6]:
class Query_process(Preprocess):
    def __init__(self):
        super().__init__()
    def __call__(self,query):
        print('原始查询语句: {}'.format(query))
        query = query.lower()
        query = nltk.word_tokenize(query)
        query = ' '.join([x for x in query if x not in self.punctuations and x not in self.stop_words or x in ['not','or','|','-']])
        query = query.replace(' or ', '|').replace(' | ','|').replace(' not ', '-').replace(' - ','-').replace(' ', '&')
        query = query.replace('|', ' | ').replace('&',' & ').replace('-',' - ')
        print('转化为表达式: {}\n'.format(query))
        query = ' '.join(['dictionary[\''+x+'\']' if x != '&' and x != '|' and x != '-' else x for x in query.split()])
        try:
            result = eval(query)
            return eval(query)
        except:
            return None

    def get_query_result_text(self,query):
        item_list = self.__call__(query)
        if item_list is None:
            return None
        result = [items[i]['text'] for i in item_list]
        result = '\n\n'.join(result)
        return result

# 显示查询交互

In [7]:
import tkinter as tk
from IPython.display import clear_output

In [8]:
win = tk.Tk()
win.title('查询')
win.geometry('700x100')

query = tk.StringVar()
entry = tk.Entry(win, relief='sunken',font='Calibri 23',width=33,textvariable=query)
entry.place(relx=0.43,rely=0.5,anchor='center')

def button_command():
    clear_output()
    query_sentence = query.get()
    print(query_process.get_query_result_text(query_sentence))

button = tk.Button(win, text='查询', height=2,width=8,command=button_command)
button.place(relx=0.9,rely=0.5,anchor='center')

In [9]:
query_process=Query_process()

In [ ]:
win.mainloop()

原始查询语句: trump
转化为表达式: trump

RT @IMAO_: Trump is a conservative like Stephen Colbert is a conservative.

GOP Grills Kerry on Iran Deal: Also, Donald Trump visits the border. http://t.co/7ZFJOYlaew #AceNewsReport

RT @NaughtyBeyotch: Donald Trump Slams Obama, John Kerry for Iran Nuclear Deal: ‘Maybe... http://t.co/EIjQz0nNFp

@cruzananddiet Do you realize that Donald Trump gave $100,000 to the CLINTON FOUNDATION ?

Not sure who's funnier: Stephen Colbert or Donald Trump.

Roswell: Not All Budget Cuts are Created Equal: Roswell CEO Donald Trump says that's $32 million the hospital an... http://bit.ly/dMu84m

Vote for Trump as a serious candidate, not this nuclear agreement stuff. @realDonaldTrump Republican GOP 2016 https://t.co/cL2O0RovmF

Dream scenario: Stephen Colbert's SuperPAC is paying Donald Trump to run for president.

Trump gave at least $100K to Clinton Foundation | TheHill http://t.co/CP8WD7Jp0S


# mak
## 带书法
#### asdf 
* sdfl
* das;fkj